## 1. Import Libraries và Load Data

In [ ]:
# Import thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

# TF-IDF và cosine similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

# Word Embedding
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

print("Libraries imported successfully!")

In [ ]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
# Load dữ liệu
movies_metadata = pd.read_csv('movies/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('movies/ratings.csv')
links = pd.read_csv('movies/links.csv')

print(f"Movies metadata shape: {movies_metadata.shape}")
print(f"Ratings shape: {ratings.shape}")
print(f"Links shape: {links.shape}")
print("\nMovies metadata sample:")
display(movies_metadata.head())

## 2. Lấy 110 phim của UserID = 201

In [ ]:
# Lọc ratings của user 201
user_201_ratings = ratings[ratings['userId'] == 201].copy()
print(f"Số lượng phim user 201 đã đánh giá: {len(user_201_ratings)}")
print("\nDữ liệu ratings của user 201:")
display(user_201_ratings.head(10))

In [ ]:
# Kiểm tra nếu user 201 có đủ 110 phim
if len(user_201_ratings) < 110:
    print(f"WARNING: User 201 chỉ có {len(user_201_ratings)} phim, ít hơn 110 phim yêu cầu!")
    print("Sẽ sử dụng tất cả phim có sẵn.")
else:
    print(f"✓ User 201 có {len(user_201_ratings)} phim (>= 110 phim)")

# Sắp xếp theo timestamp để lấy 110 phim đầu tiên
user_201_ratings = user_201_ratings.sort_values('timestamp').reset_index(drop=True)
user_201_ratings_110 = user_201_ratings.head(110).copy()

print(f"\nĐã lấy {len(user_201_ratings_110)} phim đầu tiên của user 201")

## 3. Chia tập Test (100 phim) và Profile (10 phim)

In [ ]:
# Cắt 100 phim đầu làm test set
test_set = user_201_ratings_110.head(100).copy()

# 10 phim còn lại làm profile set
profile_set = user_201_ratings_110.tail(10).copy()

print(f"Test set size: {len(test_set)}")
print(f"Profile set size: {len(profile_set)}")

print("\n=== Profile Set (10 phim để biểu diễn user 201) ===")
display(profile_set)

print("\n=== Test Set (100 phim đầu) ===")
display(test_set.head())

## 4. Mapping movieId giữa ratings.csv và movies_metadata.csv

In [ ]:
# Merge links để chuyển đổi movieId từ ratings sang tmdbId trong movies_metadata
# ratings.csv dùng movieId từ MovieLens
# movies_metadata.csv dùng id từ TMDB

# Clean movies_metadata id column (có thể có giá trị non-numeric)
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors='coerce')
movies_metadata = movies_metadata.dropna(subset=['id'])
movies_metadata['id'] = movies_metadata['id'].astype(int)

# Merge links với ratings để có tmdbId
profile_movies = profile_set.merge(links[['movieId', 'tmdbId']], on='movieId', how='left')
test_movies = test_set.merge(links[['movieId', 'tmdbId']], on='movieId', how='left')

print("Profile movies with TMDB ID:")
display(profile_movies)

# Lấy thông tin metadata cho profile movies
profile_movies_metadata = profile_movies.merge(
    movies_metadata[['id', 'title', 'overview', 'genres']], 
    left_on='tmdbId', 
    right_on='id', 
    how='left'
)

print(f"\nSố phim trong profile có metadata: {len(profile_movies_metadata.dropna(subset=['overview']))}")
display(profile_movies_metadata[['title', 'overview', 'rating']].head())

## 5. Tiền xử lý Overview

In [ ]:
# Hàm tiền xử lý text
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """Tiền xử lý text: lowercase, remove special chars, remove stopwords"""
    if not isinstance(text, str) or pd.isna(text):
        return ''
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Áp dụng preprocessing cho movies_metadata
movies_metadata['overview_clean'] = movies_metadata['overview'].apply(preprocess_text)
movies_metadata['overview_clean'] = movies_metadata['overview_clean'].fillna('')

print("Sample preprocessed overviews:")
for idx in range(3):
    print(f"\nOriginal: {movies_metadata['overview'].iloc[idx][:100]}...")
    print(f"Cleaned: {movies_metadata['overview_clean'].iloc[idx][:100]}...")

## 6. Tạo User Profile từ 10 phim

In [ ]:
# Kết hợp overview của 10 phim trong profile set để tạo user profile
profile_overviews = profile_movies_metadata.dropna(subset=['overview'])['overview'].values

# Tiền xử lý các overview này
profile_overviews_clean = [preprocess_text(overview) for overview in profile_overviews]
profile_overviews_clean = [ov for ov in profile_overviews_clean if ov]  # Remove empty

# Kết hợp tất cả thành một văn bản đại diện cho user 201
user_profile_text = ' '.join(profile_overviews_clean)

print(f"Số phim có overview trong profile: {len(profile_overviews_clean)}")
print(f"\nUser profile text length: {len(user_profile_text)} characters")
print(f"\nUser profile preview (first 500 chars):\n{user_profile_text[:500]}...")

## 7. PHƯƠNG PHÁP 1: TF-IDF Vectorization

### 7.1. Vector hóa bằng TF-IDF

In [ ]:
# Tạo TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit trên toàn bộ movies corpus
tfidf_matrix = tfidf.fit_transform(movies_metadata['overview_clean'])

print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
print(f"Number of features: {len(tfidf.get_feature_names_out())}")

In [ ]:
# Transform user profile
user_profile_tfidf = tfidf.transform([user_profile_text])

print(f"User profile TF-IDF shape: {user_profile_tfidf.shape}")
print(f"User profile TF-IDF non-zero elements: {user_profile_tfidf.nnz}")

### 7.2. Tính Cosine Similarity và Recommend Top 100

In [ ]:
# Tính cosine similarity giữa user profile và tất cả movies
cosine_similarities_tfidf = cosine_similarity(user_profile_tfidf, tfidf_matrix).flatten()

# Tạo DataFrame với similarity scores
movies_with_scores_tfidf = movies_metadata.copy()
movies_with_scores_tfidf['similarity_score'] = cosine_similarities_tfidf

# Sắp xếp theo similarity score giảm dần
movies_with_scores_tfidf = movies_with_scores_tfidf.sort_values('similarity_score', ascending=False)

print("Top 10 movies theo TF-IDF similarity:")
display(movies_with_scores_tfidf[['id', 'title', 'similarity_score']].head(10))

In [ ]:
# Lấy top 100 movies được recommend (dự đoán)
# Loại bỏ các phim đã có trong profile set
profile_movie_ids = set(profile_movies_metadata['id'].dropna().astype(int))

# Filter ra các phim không thuộc profile
movies_not_in_profile_tfidf = movies_with_scores_tfidf[
    ~movies_with_scores_tfidf['id'].isin(profile_movie_ids)
]

# Lấy top 100
top_100_recommendations_tfidf = movies_not_in_profile_tfidf.head(100)

print(f"\nĐã recommend {len(top_100_recommendations_tfidf)} phim bằng TF-IDF")
print("\nTop 10 recommendations:")
display(top_100_recommendations_tfidf[['id', 'title', 'similarity_score']].head(10))

## 8. PHƯƠNG PHÁP 2: Word Embedding (Word2Vec)

### 8.1. Tokenize và Train Word2Vec

In [ ]:
# Tokenize tất cả overviews
def tokenize_text(text):
    """Tokenize text thành list of words"""
    if not isinstance(text, str) or not text:
        return []
    return word_tokenize(text.lower())

# Tokenize toàn bộ corpus
print("Tokenizing corpus...")
tokenized_corpus = movies_metadata['overview_clean'].apply(tokenize_text).tolist()

# Remove empty lists
tokenized_corpus = [tokens for tokens in tokenized_corpus if tokens]

print(f"Number of documents: {len(tokenized_corpus)}")
print(f"Sample tokens: {tokenized_corpus[0][:20]}")

In [ ]:
# Train Word2Vec model
print("Training Word2Vec model...")
w2v_model = Word2Vec(
    sentences=tokenized_corpus,
    vector_size=100,  # Dimension of word vectors
    window=5,         # Context window size
    min_count=2,      # Ignore words with frequency < 2
    workers=4,        # Number of threads
    sg=1,             # Skip-gram model
    epochs=10
)

print(f"\nWord2Vec model trained!")
print(f"Vocabulary size: {len(w2v_model.wv)}")
print(f"Vector dimension: {w2v_model.wv.vector_size}")

### 8.2. Tạo Document Vectors

In [ ]:
def get_document_vector(tokens, model):
    """Tạo vector cho document bằng cách average word vectors"""
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    
    if not vectors:
        return np.zeros(model.wv.vector_size)
    
    return np.mean(vectors, axis=0)

# Tạo vectors cho tất cả movies
print("Creating document vectors for all movies...")
movies_metadata['tokens'] = movies_metadata['overview_clean'].apply(tokenize_text)
movies_metadata['doc_vector'] = movies_metadata['tokens'].apply(
    lambda tokens: get_document_vector(tokens, w2v_model)
)

print("Done! Sample vector shape:", movies_metadata['doc_vector'].iloc[0].shape)

In [ ]:
# Tạo user profile vector từ 10 phim
user_profile_tokens = tokenize_text(user_profile_text)
user_profile_w2v = get_document_vector(user_profile_tokens, w2v_model)

print(f"User profile W2V shape: {user_profile_w2v.shape}")
print(f"User profile W2V sample values: {user_profile_w2v[:10]}")

### 8.3. Tính Cosine Similarity và Recommend Top 100

In [ ]:
# Tạo matrix từ doc vectors
doc_vectors_matrix = np.vstack(movies_metadata['doc_vector'].values)

print(f"Document vectors matrix shape: {doc_vectors_matrix.shape}")

# Tính cosine similarity
user_profile_w2v_reshaped = user_profile_w2v.reshape(1, -1)
cosine_similarities_w2v = cosine_similarity(user_profile_w2v_reshaped, doc_vectors_matrix).flatten()

print(f"Cosine similarities shape: {cosine_similarities_w2v.shape}")
print(f"Sample similarities: {cosine_similarities_w2v[:10]}")

In [ ]:
# Tạo DataFrame với W2V similarity scores
movies_with_scores_w2v = movies_metadata.copy()
movies_with_scores_w2v['similarity_score'] = cosine_similarities_w2v

# Sắp xếp theo similarity
movies_with_scores_w2v = movies_with_scores_w2v.sort_values('similarity_score', ascending=False)

print("Top 10 movies theo Word2Vec similarity:")
display(movies_with_scores_w2v[['id', 'title', 'similarity_score']].head(10))

In [ ]:
# Lấy top 100 recommendations (loại bỏ profile movies)
movies_not_in_profile_w2v = movies_with_scores_w2v[
    ~movies_with_scores_w2v['id'].isin(profile_movie_ids)
]

top_100_recommendations_w2v = movies_not_in_profile_w2v.head(100)

print(f"\nĐã recommend {len(top_100_recommendations_w2v)} phim bằng Word2Vec")
print("\nTop 10 recommendations:")
display(top_100_recommendations_w2v[['id', 'title', 'similarity_score']].head(10))

## 9. Đánh giá: P@K, R@K, F1@K (K=100)

### 9.1. Chuẩn bị Test Set với TMDB IDs

In [ ]:
# Lấy TMDB IDs từ test set
test_movies_ids = set(test_movies['tmdbId'].dropna().astype(int))

print(f"Số phim trong test set có TMDB ID: {len(test_movies_ids)}")
print(f"Sample test movie IDs: {list(test_movies_ids)[:10]}")

### 9.2. Tính Precision@K, Recall@K, F1@K

In [ ]:
def calculate_metrics_at_k(recommended_ids, test_ids, k=100):
    """
    Tính Precision@K, Recall@K, F1@K
    
    Args:
        recommended_ids: set of recommended movie IDs
        test_ids: set of actual relevant movie IDs (ground truth)
        k: number of recommendations
    
    Returns:
        dict with precision, recall, f1
    """
    # Số phim được recommend đúng (có trong test set)
    true_positives = len(recommended_ids.intersection(test_ids))
    
    # Precision@K = TP / K
    precision = true_positives / k if k > 0 else 0
    
    # Recall@K = TP / |test set|
    recall = true_positives / len(test_ids) if len(test_ids) > 0 else 0
    
    # F1@K
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'true_positives': true_positives
    }

# Test function
print("Test metrics calculation:")
test_metrics = calculate_metrics_at_k({1,2,3}, {2,3,4,5,6}, k=5)
print(test_metrics)

In [ ]:
# Tính metrics cho TF-IDF
recommended_ids_tfidf = set(top_100_recommendations_tfidf['id'].astype(int))

metrics_tfidf = calculate_metrics_at_k(recommended_ids_tfidf, test_movies_ids, k=100)

print("=" * 60)
print("METRICS CHO TF-IDF (K=100)")
print("=" * 60)
print(f"True Positives: {metrics_tfidf['true_positives']}")
print(f"Precision@100: {metrics_tfidf['precision']:.4f}")
print(f"Recall@100: {metrics_tfidf['recall']:.4f}")
print(f"F1@100: {metrics_tfidf['f1']:.4f}")

In [ ]:
# Tính metrics cho Word2Vec
recommended_ids_w2v = set(top_100_recommendations_w2v['id'].astype(int))

metrics_w2v = calculate_metrics_at_k(recommended_ids_w2v, test_movies_ids, k=100)

print("=" * 60)
print("METRICS CHO WORD2VEC (K=100)")
print("=" * 60)
print(f"True Positives: {metrics_w2v['true_positives']}")
print(f"Precision@100: {metrics_w2v['precision']:.4f}")
print(f"Recall@100: {metrics_w2v['recall']:.4f}")
print(f"F1@100: {metrics_w2v['f1']:.4f}")

## 10. Đánh giá: MRR và NDCG

### 10.1. Sắp xếp Test Set theo Rating

In [ ]:
# Sắp xếp test set theo rating giảm dần
test_sorted = test_movies.sort_values('rating', ascending=False).reset_index(drop=True)

print("Test set đã sắp xếp theo rating:")
display(test_sorted[['movieId', 'tmdbId', 'rating', 'timestamp']].head(20))

# Tạo list các tmdbId theo thứ tự rating giảm dần
test_sorted_ids = test_sorted['tmdbId'].dropna().astype(int).tolist()
print(f"\nSố phim trong test set có TMDB ID: {len(test_sorted_ids)}")

### 10.2. Tính MRR (Mean Reciprocal Rank)

In [ ]:
def calculate_mrr(recommended_list, relevant_list):
    """
    Tính Mean Reciprocal Rank (MRR)
    
    Args:
        recommended_list: list of recommended movie IDs (theo thứ tự)
        relevant_list: list of relevant movie IDs
    
    Returns:
        MRR score
    """
    relevant_set = set(relevant_list)
    
    for rank, movie_id in enumerate(recommended_list, start=1):
        if movie_id in relevant_set:
            return 1.0 / rank
    
    return 0.0

# Test
test_mrr = calculate_mrr([1, 2, 3, 4, 5], [3])
print(f"Test MRR (relevant at position 3): {test_mrr}")

In [ ]:
# Tính MRR cho TF-IDF
recommended_list_tfidf = top_100_recommendations_tfidf['id'].astype(int).tolist()
mrr_tfidf = calculate_mrr(recommended_list_tfidf, test_sorted_ids)

print(f"MRR for TF-IDF: {mrr_tfidf:.4f}")

In [ ]:
# Tính MRR cho Word2Vec
recommended_list_w2v = top_100_recommendations_w2v['id'].astype(int).tolist()
mrr_w2v = calculate_mrr(recommended_list_w2v, test_sorted_ids)

print(f"MRR for Word2Vec: {mrr_w2v:.4f}")

### 10.3. Tính NDCG (Normalized Discounted Cumulative Gain)

In [ ]:
def calculate_dcg(recommended_list, relevant_list, k=100):
    """
    Tính DCG@K
    Relevance score = 1 nếu có trong relevant_list, 0 nếu không
    """
    relevant_set = set(relevant_list)
    dcg = 0.0
    
    for i, movie_id in enumerate(recommended_list[:k], start=1):
        if movie_id in relevant_set:
            # Relevance = 1 for relevant items
            dcg += 1.0 / np.log2(i + 1)
    
    return dcg

def calculate_ndcg(recommended_list, relevant_list, k=100):
    """
    Tính NDCG@K
    
    Args:
        recommended_list: list of recommended movie IDs
        relevant_list: list of relevant movie IDs (đã sắp xếp theo rating)
        k: cutoff position
    
    Returns:
        NDCG score
    """
    # DCG của recommendation
    dcg = calculate_dcg(recommended_list, relevant_list, k)
    
    # IDCG (ideal DCG) - nếu recommend đúng hết các relevant items
    idcg = calculate_dcg(relevant_list, relevant_list, k)
    
    if idcg == 0:
        return 0.0
    
    return dcg / idcg

# Test
test_ndcg = calculate_ndcg([1, 2, 3, 4, 5], [3, 1], k=5)
print(f"Test NDCG: {test_ndcg:.4f}")

In [ ]:
# Tính NDCG cho TF-IDF
ndcg_tfidf = calculate_ndcg(recommended_list_tfidf, test_sorted_ids, k=100)

print(f"NDCG@100 for TF-IDF: {ndcg_tfidf:.4f}")

In [ ]:
# Tính NDCG cho Word2Vec
ndcg_w2v = calculate_ndcg(recommended_list_w2v, test_sorted_ids, k=100)

print(f"NDCG@100 for Word2Vec: {ndcg_w2v:.4f}")

## 11. So sánh và Kết luận

In [ ]:
# Tạo bảng so sánh
comparison_df = pd.DataFrame({
    'Metric': ['Precision@100', 'Recall@100', 'F1@100', 'MRR', 'NDCG@100'],
    'TF-IDF': [
        f"{metrics_tfidf['precision']:.4f}",
        f"{metrics_tfidf['recall']:.4f}",
        f"{metrics_tfidf['f1']:.4f}",
        f"{mrr_tfidf:.4f}",
        f"{ndcg_tfidf:.4f}"
    ],
    'Word2Vec': [
        f"{metrics_w2v['precision']:.4f}",
        f"{metrics_w2v['recall']:.4f}",
        f"{metrics_w2v['f1']:.4f}",
        f"{mrr_w2v:.4f}",
        f"{ndcg_w2v:.4f}"
    ]
})

print("\n" + "="*70)
print("BẢNG SO SÁNH KẾT QUẢ: TF-IDF vs WORD2VEC")
print("="*70)
display(comparison_df)

In [ ]:
# Visualization
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: P@K, R@K, F1@K
metrics_names = ['Precision@100', 'Recall@100', 'F1@100']
tfidf_scores = [metrics_tfidf['precision'], metrics_tfidf['recall'], metrics_tfidf['f1']]
w2v_scores = [metrics_w2v['precision'], metrics_w2v['recall'], metrics_w2v['f1']]

x = np.arange(len(metrics_names))
width = 0.35

axes[0].bar(x - width/2, tfidf_scores, width, label='TF-IDF', alpha=0.8)
axes[0].bar(x + width/2, w2v_scores, width, label='Word2Vec', alpha=0.8)
axes[0].set_ylabel('Score')
axes[0].set_title('Precision, Recall, F1 @ K=100')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics_names)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Plot 2: MRR và NDCG
ranking_metrics = ['MRR', 'NDCG@100']
tfidf_ranking = [mrr_tfidf, ndcg_tfidf]
w2v_ranking = [mrr_w2v, ndcg_w2v]

x2 = np.arange(len(ranking_metrics))

axes[1].bar(x2 - width/2, tfidf_ranking, width, label='TF-IDF', alpha=0.8)
axes[1].bar(x2 + width/2, w2v_ranking, width, label='Word2Vec', alpha=0.8)
axes[1].set_ylabel('Score')
axes[1].set_title('Ranking Metrics')
axes[1].set_xticks(x2)
axes[1].set_xticklabels(ranking_metrics)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Nhận xét và Kết luận

### Nhận xét:

#### 1. **So sánh hiệu suất:**

**TF-IDF:**
- **Ưu điểm:**
  - Nhanh và hiệu quả về mặt tính toán
  - Dễ hiểu và triển khai
  - Hiệu quả với dữ liệu có cấu trúc rõ ràng
  - Bắt được các từ khóa quan trọng và đặc trưng

- **Nhược điểm:**
  - Không bắt được ý nghĩa ngữ nghĩa (semantic meaning)
  - Không xử lý được từ đồng nghĩa
  - Ma trận thưa (sparse matrix) có thể dẫn đến mất thông tin

**Word2Vec:**
- **Ưu điểm:**
  - Bắt được mối quan hệ ngữ nghĩa giữa các từ
  - Xử lý tốt từ đồng nghĩa và tương tự
  - Vector dense (không thưa) giữ được nhiều thông tin
  - Có khả năng generalize tốt hơn

- **Nhược điểm:**
  - Yêu cầu corpus lớn để train tốt
  - Phức tạp và tốn thời gian hơn
  - Có thể không hiệu quả với corpus nhỏ

#### 2. **Kết quả thực nghiệm:**

Dựa vào các metrics:
- **Precision@100, Recall@100, F1@100**: So sánh khả năng tìm đúng phim trong test set
- **MRR**: Đánh giá vị trí của phim relevant đầu tiên
- **NDCG@100**: Đánh giá chất lượng ranking tổng thể

#### 3. **Kết luận:**

- Nếu TF-IDF có metrics cao hơn → phù hợp hơn cho bài toán này với dataset có sẵn
- Nếu Word2Vec tốt hơn → khả năng bắt semantic similarity tốt hơn cho recommend
- Có thể kết hợp cả 2 phương pháp (ensemble) để tận dụng ưu điểm của mỗi phương pháp

#### 4. **Đề xuất cải thiện:**

1. **Hybrid approach**: Kết hợp TF-IDF và Word2Vec với weight
2. **Fine-tune Word2Vec**: Điều chỉnh hyperparameters (vector_size, window, min_count)
3. **Thêm features**: Kết hợp thêm genres, keywords, actors
4. **Ensemble learning**: Sử dụng voting hoặc stacking
5. **Deep Learning**: Sử dụng BERT, Doc2Vec hoặc neural embeddings

In [ ]:
# In summary cuối cùng
print("\n" + "="*70)
print("TÓM TẮT KẾT QUẢ THỰC NGHIỆM")
print("="*70)
print(f"\n1. User ID: 201")
print(f"2. Tổng số phim đã đánh giá: {len(user_201_ratings)}")
print(f"3. Số phim sử dụng: 110")
print(f"   - Test set: 100 phim")
print(f"   - Profile set: 10 phim")
print(f"\n4. Phương pháp đánh giá:")
print(f"   - Content-Based Filtering với TF-IDF")
print(f"   - Content-Based Filtering với Word2Vec")
print(f"\n5. Metrics sử dụng:")
print(f"   - Precision@100, Recall@100, F1@100")
print(f"   - MRR (Mean Reciprocal Rank)")
print(f"   - NDCG@100 (Normalized Discounted Cumulative Gain)")

print(f"\n6. Kết quả:")
print(f"\n   TF-IDF:")
print(f"   - True Positives: {metrics_tfidf['true_positives']}")
print(f"   - Precision@100: {metrics_tfidf['precision']:.4f}")
print(f"   - Recall@100: {metrics_tfidf['recall']:.4f}")
print(f"   - F1@100: {metrics_tfidf['f1']:.4f}")
print(f"   - MRR: {mrr_tfidf:.4f}")
print(f"   - NDCG@100: {ndcg_tfidf:.4f}")

print(f"\n   Word2Vec:")
print(f"   - True Positives: {metrics_w2v['true_positives']}")
print(f"   - Precision@100: {metrics_w2v['precision']:.4f}")
print(f"   - Recall@100: {metrics_w2v['recall']:.4f}")
print(f"   - F1@100: {metrics_w2v['f1']:.4f}")
print(f"   - MRR: {mrr_w2v:.4f}")
print(f"   - NDCG@100: {ndcg_w2v:.4f}")

# Xác định phương pháp tốt hơn
if metrics_tfidf['f1'] > metrics_w2v['f1']:
    winner = "TF-IDF"
    winner_f1 = metrics_tfidf['f1']
elif metrics_w2v['f1'] > metrics_tfidf['f1']:
    winner = "Word2Vec"
    winner_f1 = metrics_w2v['f1']
else:
    winner = "Ngang nhau"
    winner_f1 = metrics_tfidf['f1']

print(f"\n" + "="*70)
print(f"KẾT LUẬN: Phương pháp {winner} cho kết quả tốt hơn với F1@100 = {winner_f1:.4f}")
print("="*70)